In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes
!pip install -q sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.5 MB/s eta 0:00:00
,

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
import json

file_path = '/content/drive/My Drive/knowledgebase.json'

# ✅ Load the whole JSON array at once
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)  # 👈 this is the key fix

instruct_data = []

for item in data:
    title = item.get("title", "")
    summary = item.get("summary", "")
    key_principles = item.get("key_principles", [])
    tags = item.get("tags", [])
    personality_links = item.get("personality_links", [])
    transformation = item.get("positive_transformation", "")
    section = item.get("section", "")

    # 1. General law explanation
    instruct_data.append({
        "instruction": f"What is the law of {title}?",
        "input": "",
        "output": f"{summary} This falls under the section: {section}. Key principles include: {'; '.join(key_principles)}"
    })

    # 2. Emotionally personalized question
    if transformation:
        instruct_data.append({
            "instruction": f"I struggle with {', '.join(tags[:2])}. How can I improve?",
            "input": "",
            "output": f"According to the law of {title}, here's how to transform: {transformation}"
        })

    # 3. Personality-driven pairing
    for p in personality_links:
        instruct_data.append({
            "instruction": f"What should someone with {p} personality understand about human nature?",
            "input": "",
            "output": f"The law of {title} offers insights. Summary: {summary}. Key advice: {'; '.join(key_principles)}"
        })

    # 4. Tag-based advice
    if tags:
        instruct_data.append({
            "instruction": f"What law applies when dealing with {tags[0]}?",
            "input": "",
            "output": f"{title}: {summary}. Here's how to apply it: {'; '.join(key_principles)}"
        })

# ✅ Save as JSONL for fine-tuning
with open("/content/human_nature_instruct.jsonl", "w", encoding='utf-8') as f:
    for ex in instruct_data:
        f.write(json.dumps(ex) + "\n")

print(f"✅ Generated {len(instruct_data)} instruction-style training examples.")


✅ Generated 205 instruction-style training examples.


In [4]:
!pip install -q faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 44.4 MB/s eta 0:00:00
,

In [6]:
import json

# Load the JSONL file
file_path = "/content/human_nature_instruct.jsonl"
dataset = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        dataset.append(item)

print(f"✅ Loaded {len(dataset)} items")


✅ Loaded 205 items


In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Choose a sentence transformer model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare texts to embed (instruction + input)
texts = [f"{ex['instruction']} {ex['input']}" for ex in dataset]

# Generate embeddings
embeddings = embedder.encode(texts, convert_to_numpy=True)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Map index to outputs
id_to_output = {i: dataset[i]["output"] for i in range(len(dataset))}


,/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
,The secret `HF_TOKEN` does not exist in your Colab secrets.
,To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
,You will be able to reuse this secret in all of your notebooks.
,Please note that authentication is recommended but still optional to access public models or datasets.
,  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
def ask_question(query, top_k=1):
    query_vec = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_vec, top_k)
    responses = [id_to_output[i] for i in I[0]]
    return responses

# 🔍 Try a test query
query1 ="Explain the concept of 'The Shadow' and how we should interact with it?"
responses = ask_question(query)
print("🤖 Answer:")
print(responses[0])
query2 = "How does the story of Pericles illustrate the Law of Irrationality?"
responses = ask_question(query2)
print("🤖 Answer:")
print(responses[0])

🤖 Answer:
,This law asserts that our attitude—the lens through which we interpret the world—actively shapes our circumstances. A fearful or hostile attitude leads to self-sabotage by unconsciously creating the negative outcomes we expect. The story of Anton Chekhov's transformation from a brutalized child to a man of great empathy demonstrates that we have the power to consciously change our attitude. By adopting an expansive, positive, and tolerant outlook, we can overcome adversity, create opportunities, and break free from self-imposed limitations, thus altering our destiny. This falls under the section: Chapter 8. Key principles include: Your attitude creates a self-fulfilling dynamic; a negative mindset creates negative results, while a positive one opens up possibilities.; You are not a pawn of circumstance; you can fundamentally alter your life by consciously altering your attitude.; Recognize and overcome constricted (negative) attitudes like hostility, anxiety, avoidance, depr